In [243]:
%load_ext autoreload
%autoreload complete

import sys
import importlib
import warnings

importlib.invalidate_caches()

for mod_name in list(sys.modules):
    if not (str.startswith(mod_name, "robotodo") or str.startswith(mod_name, "tensorspecs")):
        continue
    try:
        importlib.reload(sys.modules[mod_name])
    except Exception as error:
        warnings.warn(f"{error}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# TODO 
from robotodo.engines.isaac.builders import load_usd_scene

# TODO set up axis
scene = await load_usd_scene(
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Simple_Room/simple_room.usd",
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/default_environment.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/gridroom_black.usd",
)

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'

|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------

2025-11-13T07:56:01Z [12,254ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed
2025-11-13T07:56:02Z [13,356ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-11-13T07:56:02Z [13,356ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-11-13T07:56:02Z [13,455ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


In [3]:
list(scene.traverse())

['/',
 '/OmniverseKit_Right',
 '/OmniverseKit_Front',
 '/World',
 '/World/Looks',
 '/World/Looks/theGrid',
 '/World/Looks/theGrid/Shader',
 '/World/GroundPlane',
 '/World/GroundPlane/CollisionPlane',
 '/World/SphereLight',
 '/World/Environment',
 '/World/Environment/Geometry',
 '/OmniverseKit_Top',
 '/Render',
 '/Render/OmniverseKit',
 '/Render/OmniverseKit/HydraTextures',
 '/Render/OmniverseKit/HydraTextures/omni_kit_widget_viewport_ViewportTexture_0',
 '/Render/OmniverseGlobalRenderSettings',
 '/Render/Vars',
 '/Render/Vars/LdrColor',
 '/OmniverseKit_Persp']

In [4]:
from robotodo.engines.isaac.scene import Scene
from robotodo.engines.isaac.builders import load_urdf, load_usd, builder, EntityBodyKind
from robotodo.utils.pose import Pose


# scene = Scene(_kernel=kernel)


# TODO this requires the stage to be editable!!!!
panda = await load_urdf(
    "./todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf", 
    scene=scene, 
    fix_root_link=True,
)


mug = await load_usd(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Props/Mugs/SM_Mug_A2.usd",
    scene=scene,
)
builder.reset(mug, body_kind=EntityBodyKind.RIGID)

mug.pose = Pose(p=[.5, .5, .5])


In [5]:
ant = await load_usd(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Robots/IsaacSim/Ant/ant_instanceable.usd",
    scene=scene,
    path="/Ant",
    replace=True,
)

2025-11-13T07:56:07Z [18,459ms] [Error] [omni.usd.commands.usd_commands] /Ant does not exist


In [6]:
from robotodo.engines.isaac.articulation import Articulation

ant = Articulation("/Ant", scene=scene)

In [7]:
ant._isaac_physics_articulation_view.shared_metatype.joint_names

['front_left_leg',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg',
 'front_left_foot',
 'front_right_foot',
 'left_back_foot',
 'right_back_foot']

In [8]:
ant._isaac_physics_articulation_view.shared_metatype.joint_types

[<JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>]

In [9]:
ant._isaac_physics_articulation_view.shared_metatype.link_names

['torso',
 'front_left_leg',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg',
 'front_left_foot',
 'front_right_foot',
 'left_back_foot',
 'right_back_foot']

In [10]:
(ant._isaac_physics_articulation_view.shared_metatype.link_parents)


['',
 'torso',
 'torso',
 'torso',
 'torso',
 'front_left_leg',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg']

In [11]:
ant.joints["front_left_leg"].pose_in_body0

Pose(p=array([[0.2, 0.2, 0. ]]), q=array([[ 0.        , -0.70710683,  0.        ,  0.70710683]]))

In [12]:
for joint_name, joint in ant.joints.items():
    # TODO
    joint.path
    joint.body0.path, joint.body1.path
    print(joint.pose_in_body0 * joint.pose_in_body1.inv())

Pose(p=array([[0.2, 0.2, 0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[-0.2,  0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[-0.2, -0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[ 0.2, -0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[0.2, 0.2, 0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[-0.2,  0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[-0.2, -0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))
Pose(p=array([[ 0.2, -0.2,  0. ]]), q=array([[0., 0., 0., 1.]]))


In [13]:
ant._isaac_physics_articulation_view.shared_metatype.link_parents

['',
 'torso',
 'torso',
 'torso',
 'torso',
 'front_left_leg',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg']

In [ ]:
# TODO
import functools
import warnings
from typing import Callable, TypedDict

import numpy
from curobo.cuda_robot_model.kinematics_parser import (
    JointType as CuroboJointType,
    KinematicsParser as CuroboKinematicsParser,
    LinkParams as CuroboLinkParams,
)
# TODO use core
from robotodo.engines.isaac.entity import Entity
from robotodo.engines.isaac.articulation import (
    Axis,
    Joint, 
    JointKind, 
    FixedJoint,
    RevoluteJoint, 
    PrismaticJoint,
)



import numpy


# TODO
def _array_reduce_single(x: numpy.ndarray, shape: tuple[int, ...]):
    x = numpy.reshape(x, (-1, *shape))
    x = numpy.unique(x, axis=0)
    try:
        x = numpy.squeeze(x, axis=0)
    except Exception as error:
        # TODO better msg
        raise ValueError(
            f"Failed to reduce batched array from "
            f"shape {x.shape} to {shape}, "
            f"are the elements along the leading batch dimension unique?: {x}"
        ) from error
    return x

# TODO
def test_array_reduce_single():
    # TODO
    _array_reduce_single(["a", "a"], shape=())
    _array_reduce_single(numpy.full((2, 3, 4), fill_value=1), shape=(3, 4))
    _array_reduce_single(numpy.random.rand(*(2, 3, 4)), shape=(3, 4))


class _CuroboKinematicsParser(CuroboKinematicsParser):
    # TODO rm
    # class Config(TypedDict):
    #     joints: list[Joint]
    #     joint_name_fn: Callable[[Joint], str] | None
    #     link_name_fn: Callable[[Entity], str] | None

    def __init__(
        self, 
        # TODO ProtoJoint
        robotodo_joints: list[Joint],
        extra_links: dict[str, CuroboLinkParams] | None = None,
    ):
        # TODO do something
        self._robotodo_joints = robotodo_joints
        super().__init__(extra_links=extra_links)

    # TODO
    def _robotodo_derive_entity_name(self, entity: Joint | Entity):
        if entity.label is not None:
            return _array_reduce_single(entity.label, shape=()).item()
        # TODO is this the only way to make it hashable???
        return str(entity.path)

    # TODO NOTE override
    def build_link_parent(self):
        for joint in self._robotodo_joints:
            # TODO rm
            # parent_link_name = joint.body0.path[0]
            # child_link_name = joint.body1.path[0]
            parent_link_name = self._robotodo_derive_entity_name(joint.body0)
            child_link_name = self._robotodo_derive_entity_name(joint.body1)            
            self._parent_map[child_link_name] = {"parent": parent_link_name}
    
    # TODO
    @functools.lru_cache
    def _build_link_parameters(self) -> dict[str, CuroboLinkParams]:
        link_params = dict()

        # TODO
        for joint in self._robotodo_joints:
            # TODO rm
            # TODO relative to articulation?
            # parent_link_name = joint.body0.path[0]
            # link_name = joint.body1.path[0]
            parent_link_name = self._robotodo_derive_entity_name(joint.body0)
            link_name = self._robotodo_derive_entity_name(joint.body1)     

            # TODO rm
            # joint_name = joint.path[0]
            joint_name = self._robotodo_derive_entity_name(joint)
            joint_limits = None

            # TODO !!!!!
            joint_type = CuroboJointType.FIXED
            match _array_reduce_single(joint.kind, shape=()):
                case JointKind.FIXED:
                    # TODO
                    joint_type = CuroboJointType.FIXED
                case JointKind.REVOLUTE:
                    # TODO
                    revolute_joint = RevoluteJoint(joint)
                    # TODO
                    match _array_reduce_single(revolute_joint.axis, shape=()):
                        case Axis.X:
                            joint_type = CuroboJointType.X_ROT
                        case Axis.Y:
                            joint_type = CuroboJointType.Y_ROT
                        case Axis.Z:
                            joint_type = CuroboJointType.Z_ROT
                        case _:
                            # TODO
                            raise ValueError("TODO")
                    # TODO use common range instead?
                    joint_limits = _array_reduce_single(
                        revolute_joint.position_limit,
                        shape=(2, ),
                    )
                case JointKind.PRISMATIC:
                    prismatic_joint = PrismaticJoint(joint)
                    match _array_reduce_single(prismatic_joint.axis, shape=()):
                        case Axis.X:
                            joint_type = CuroboJointType.X_PRISM
                        case Axis.Y:
                            joint_type = CuroboJointType.Y_PRISM
                        case Axis.Z:
                            joint_type = CuroboJointType.Z_PRISM
                        case _:
                            # TODO
                            raise ValueError("TODO")
                    # TODO use common range instead?
                    joint_limits = _array_reduce_single(
                        prismatic_joint.position_limit,
                        shape=(2, ),
                    )
                case _:
                    joint_type = CuroboJointType.FIXED
                    warnings.warn(f"Unsupported joint kind {joint.kind}, planning as fixed: {joint}")

            joint_frame = _array_reduce_single(
                (joint.pose_in_body0 * joint.pose_in_body1.inv()).to_matrix(),
                # (joint.pose_in_body1[0].inv() * joint.pose_in_body0[0])
                shape=(4, 4),
            )
            # TODO ensure homogenous

            # import numpy
            # joint_frame = numpy.linalg.inv(
            #     joint.pose_in_body1[0].to_matrix()
            # ) @ joint.pose_in_body0[0].to_matrix()

            # TODO !!!
            # print("joint origin", joint_name, (joint.pose_in_body1[0].inv() * joint.pose_in_body0[0]))
            # print("", joint_name, joint_frame)

            link_params[link_name] = CuroboLinkParams(
                link_name=link_name,
                joint_name=joint_name,
                joint_type=joint_type,
                fixed_transform=joint_frame,
                parent_link_name=parent_link_name,
                # TODO
                joint_limits=joint_limits,
            )

        return link_params

    def get_link_parameters(self, link_name: str, base: bool = False) -> CuroboLinkParams:
        # TODO
        if base:
            return CuroboLinkParams(
                link_name=link_name,
                # TODO !!!! ensure unique
                joint_name="",
                joint_type=CuroboJointType.FIXED,
                fixed_transform=numpy.eye(4),
            )
        # TODO !!!!!
        return self._build_link_parameters()[link_name]


In [ ]:
class TestCuroboKinematicsParser:
    def test_todo(self):
        pass

In [320]:
from robotodo.engines.isaac.articulation import Articulation

panda = Articulation("/_01", scene=scene)

In [321]:
from curobo.cuda_robot_model.urdf_kinematics_parser import UrdfKinematicsParser as CuroboUrdfKinematicsParser

# TODO
urdf_kinematics_parser = CuroboUrdfKinematicsParser(urdf_path="./todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf")
kinematics_parser = _CuroboKinematicsParser(
    list(panda.joints.values())
)

In [322]:
import numpy


# TODO test
parent_map_desired = urdf_kinematics_parser._parent_map
parent_map_actual = kinematics_parser._parent_map

assert {
    (key, value["parent"])
    for key, value in parent_map_desired.items()
} == {
    (key, value["parent"])
    for key, value in parent_map_actual.items()
}


for link_name, base in (
    ("base_link", True),
    ("panda_link0", False),
    ("panda_link1", False),
    ("panda_link2", False),
    ("panda_link3", False),
    ("panda_link4", False),
    ("panda_link5", False),
    ("panda_link6", False),
    ("panda_link7", False),
    ("panda_link8", False),
    ("panda_hand", False),
    ("ee_link", False),
    ("panda_leftfinger", False),
    ("panda_rightfinger", False),
    ("right_gripper", False),
):
    # TODO
    urdf_link_params = urdf_kinematics_parser.get_link_parameters(link_name=link_name, base=base)
    link_params = kinematics_parser.get_link_parameters(link_name=f"{link_name}", base=base)

    assert urdf_link_params.joint_type == link_params.joint_type
    numpy.testing.assert_almost_equal(
        desired=urdf_link_params.fixed_transform, 
        actual=link_params.fixed_transform,
        decimal=3,
        err_msg=f"TODO {link_name} {base}",
    )

    joint_limits_desired = urdf_link_params.joint_limits
    joint_limits_actual = link_params.joint_limits
    if any(x is None for x in (joint_limits_desired, joint_limits_actual)):
        numpy.testing.assert_equal(
            desired=joint_limits_desired,
            actual=joint_limits_actual,
        )
    else:
        numpy.testing.assert_almost_equal(
            desired=joint_limits_desired,
            actual=joint_limits_actual,
            decimal=3,
        )




In [323]:
kinematics_parser.get_link_parameters("panda_link0")

LinkParams(link_name='panda_link0', joint_name='panda_fixed', joint_type=<JointType.FIXED: -1>, fixed_transform=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), parent_link_name='base_link', joint_limits=None, joint_axis=None, joint_id=None, joint_velocity_limits=[-2.0, 2.0], joint_offset=[1.0, 0.0], mimic_joint_name=None)

In [324]:
kinematics_parser._parent_map

{'panda_link0': {'parent': 'base_link'},
 'panda_link1': {'parent': 'panda_link0'},
 'panda_link2': {'parent': 'panda_link1'},
 'panda_link3': {'parent': 'panda_link2'},
 'panda_link4': {'parent': 'panda_link3'},
 'panda_link5': {'parent': 'panda_link4'},
 'panda_link6': {'parent': 'panda_link5'},
 'panda_link7': {'parent': 'panda_link6'},
 'panda_link8': {'parent': 'panda_link7'},
 'right_gripper': {'parent': 'panda_link7'},
 'panda_hand': {'parent': 'panda_link8'},
 'ee_link': {'parent': 'panda_hand'},
 'panda_leftfinger': {'parent': 'panda_hand'},
 'panda_rightfinger': {'parent': 'panda_hand'}}

In [325]:
kinematics_parser.get_chain

<bound method KinematicsParser.get_chain of <__main__._CuroboKinematicsParser object at 0x7fae41a2cf50>>

In [326]:
kinematics_parser

In [366]:
import functools
from typing import TypedDict, Unpack

import einops
import torch
from tensorspecs import TensorSpec, TensorTableSpec, TensorTableLike
from curobo.types.robot import RobotConfig as CuroboRobotConfig
from curobo.types.math import Pose as CuroboPose
from curobo.types.robot import JointState as CuroboJointState
from curobo.cuda_robot_model.cuda_robot_generator import (
    CudaRobotGenerator,
    CudaRobotGeneratorConfig as CuroboCudaRobotGeneratorConfig,
)
from curobo.cuda_robot_model.cuda_robot_model import (
    CudaRobotModelConfig as CuroboCudaRobotModelConfig,
)
from curobo.wrap.reacher.motion_gen import (
    MotionGen as CuroboMotionGen,
    MotionGenConfig as CuroboMotionGenConfig,
)


class MotionPlanningError(Exception):
    pass


class MotionPlanner:
    class Config(TypedDict):
        base_link: str
        end_link: str

    # TODO
    def __init__(
        self, 
        # TODO ArticulationKinematics instead
        target: Articulation | list[Joint], 
        config: Config = Config(),
        **config_kwds: Unpack[Config],
    ):
        match target:
            case Articulation() as articulation:
                # TODO
                curobo_kinematics_parser = (
                    _CuroboKinematicsParser(
                        list(articulation.joints.values())
                    )
                )
            case list() as joints:
                # TODO
                curobo_kinematics_parser = (
                    _CuroboKinematicsParser(joints)        
                )
            case _:
                # TODO
                raise NotImplementedError("TODO")
        
        config = MotionPlanner.Config(config, **config_kwds)

        # TODO https://github.com/NVlabs/curobo/blob/ebb71702f3f70e767f40fd8e050674af0288abe8/src/curobo/cuda_robot_model/cuda_robot_generator.py#L319
        self._curobo_motion_gen = CuroboMotionGen(
            CuroboMotionGenConfig.load_from_robot_config(
                robot_cfg=CuroboRobotConfig(
                    kinematics=CuroboCudaRobotModelConfig.from_config(
                        CuroboCudaRobotGeneratorConfig(
                            # TODO infer from dof_names
                            base_link=config["base_link"],
                            ee_link=config["end_link"],
                            custom_kinematics_parser=curobo_kinematics_parser,
                        ),
                    ),
                ),
            ),
        )

    # TODO rm #############
    # @property
    # def link_labels(self):
    #     link_labels = set()
    #     for key, value in self._curobo_kinematics_parser._parent_map.items():
    #         link_labels.add(key)
    #         link_labels.add(value["parent"])
    #     return link_labels

    # # TODO !!!!
    # @property
    # def joint_labels(self):
    #     raise NotImplementedError
    
    # @property
    # def base_link_label(self):
    #     # TODO
    #     return self.__base_link_label

    # @base_link_label.setter
    # def base_link_label(self, value: ...):
    #     self.__base_link_label = value

    # @property
    # def end_link_label(self):
    #     return self.__end_link_label

    # @end_link_label.setter
    # def end_link_label(self, value: ...):
    #     self.__end_link_label = value
    # TODO rm #############

    @property
    def base_link(self):
        return self._curobo_motion_gen.kinematics.base_link

    @property
    def end_link(self):
        return self._curobo_motion_gen.kinematics.ee_link

    @property
    def joints(self):
        return self._curobo_motion_gen.joint_names
    
    @property
    def dof_names(self):
        return self._curobo_motion_gen.joint_names
    
    @property
    def observation_spec(self):
        n_dofs = len(self.dof_names)
        # TODO named indexing
        return TensorTableSpec({
            "dof_positions": TensorSpec("n? dof", shape={"dof": n_dofs}),
            # "target_pose": PoseSpec("n?"),
            # "target_pose_candidates": PoseSpec("n? candidate"),
            # "obstacles": ...,
        })
        
    @property
    def action_spec(self):
        n_dofs = len(self.dof_names)
        return TensorTableSpec({
            # TODO -or- namedtensor??
            # "dof_names": TensorSpec("dof"),
            "dof_positions": TensorSpec("time n? dof", shape={"dof": n_dofs}),
            "dof_velocities": TensorSpec("time n? dof", shape={"dof": n_dofs}),
        })

    def compute_action(
        self, 
        observation: TensorTableLike[observation_spec],
    ) -> TensorTableLike[action_spec]:
        """
        TODO doc

        """

        # TODO
        dof_positions = observation["dof_positions"]
        target_pose: Pose = observation["target_pose"]

        dof_positions, batch_shapes = einops.pack([dof_positions], "* dof")

        # TODO
        target_pose_p, _ = einops.pack([target_pose.p], "* p")
        target_pose_q, _ = einops.pack([target_pose.q], "* q")

        # TODO broadcast target_pose?? necesito??
        # einops.broadcast
        # einops.unpack(target_pose.p, batch_shapes, "* p")
        # einops.unpack(target_pose.q, batch_shapes, "* q")

        try:
            # TODO .plan_batch_env for batch collisions
            plan_result = self._curobo_motion_gen.plan_batch(
                start_state=CuroboJointState.from_position(
                    position=torch.as_tensor(dof_positions, dtype=torch.float32).cuda(),
                ),
                goal_pose=CuroboPose(
                    position=torch.as_tensor(target_pose_p, dtype=torch.float32).cuda(), 
                    # TODO use named wxyz
                    quaternion=torch.as_tensor(target_pose_q, dtype=torch.float32).cuda()
                        [..., [3, 0, 1, 2]],
                ),
                # plan_config=MotionGenPlanConfig(num_graph_seeds=1, enable_graph=False, max_attempts=100),
                # plan_config=MotionGenPlanConfig(
                #     # num_graph_seeds=1,
                #     # TODO this doesnt seem to do anything
                #     # check_start_validity=False
                # ),
            )
        except Exception as error:
            raise RuntimeError(
                f"An error occurred while planning: are the DOF positions valid?"
            ) from error

        # TODO
        # plan_result.interpolated_plan.position
        # plan_result.interpolated_plan.velocity

        if plan_result.optimized_plan is None:
            raise MotionPlanningError(f"Failed to generate optimized plan: {plan_result.status}")

        if plan_result.success is None or not torch.all(plan_result.success):
            warnings.warn(f"Motion planning is partially successful: {plan_result.status}")

        dof_positions_result = plan_result.optimized_plan.position
        # NOTE curobo auto-squeezes the result which must be undone!
        dof_positions_result = dof_positions_result.reshape(
            -1, *dof_positions_result.shape[-2:],
        )
        [dof_positions_result] = einops.unpack(
            dof_positions_result,
            batch_shapes, "* time dof",
        )

        dof_velocities_result = plan_result.optimized_plan.velocity
        # NOTE curobo auto-squeezes the result which must be undone!
        dof_velocities_result = dof_velocities_result.reshape(
            -1, *dof_velocities_result.shape[-2:],
        )
        # TODO
        [dof_velocities_result] = einops.unpack(
            dof_velocities_result,
            batch_shapes, "* time dof",
        )

        action = {
            "dof_names": self.dof_names,
            "dof_positions": dof_positions_result,
            "dof_velocities": dof_velocities_result,
        }

        return action



In [367]:
# TODO
import functools


# TODO mv
ArticulationAction = TensorTableLike[{
    # TODO -or- namedtensor??
    "dof_names": TensorSpec("dof"),
    "dof_positions": TensorSpec("n? time dof"),
    "dof_velocities": TensorSpec("n? time dof"),
}]
"""TODO doc articulation action protocol type"""


class ArticulationPlanner:
    class Config(TypedDict):
        base_link: str
        end_link: str

    # TODO
    def __init__(
        self, 
        articulation: Articulation, 
        config: Config = Config(),
        **config_kwds: Unpack[Config],
    ):
        self._articulation = articulation
        self._config = ArticulationPlanner.Config(config, **config_kwds)

    # TODO
    @functools.cached_property
    def _motion_planner(self):
        return MotionPlanner(
            # TODO articulation.kinematics
            self._articulation,
            config=MotionPlanner.Config(
                base_link=self._config["base_link"],
                end_link=self._config["end_link"],
            ),
        )
    
    @property
    def observation_spec(self):
        n_dofs = len(self._motion_planner.dof_names)
        # TODO named indexing
        return TensorTableSpec({
            # TODO also dof_names???
            "dof_positions": TensorSpec("n? dof", shape={"dof": n_dofs}),
            # "target_pose": PoseSpec("n?"),
            # "target_pose_from_base": PoseSpec("n?"),
            # "target_pose_candidates": PoseSpec("n? candidate"),
            # "target_pose_from_base_candidates": PoseSpec("n? candidate"),
            # "obstacles": ...,
        })
        
    @property
    def action_spec(self):
        n_dofs = len(self._motion_planner.dof_names)
        return TensorTableSpec({
            # TODO -or- namedtensor??
            "dof_names": TensorSpec("dof", shape={"dof": n_dofs}),
            "dof_positions": TensorSpec("n? time dof", shape={"dof": n_dofs}),
            "dof_velocities": TensorSpec("n? time dof", shape={"dof": n_dofs}),
        })

    # TODO
    def compute_action(
        self, 
        observation: TensorTableLike[observation_spec],
    ) -> TensorTableLike[action_spec] | ArticulationAction:
        
        dof_positions = observation.get("dof_positions", None)
        if dof_positions is None:
            # TODO cache index
            dof_positions = self._articulation.dof_positions[..., [
                self._articulation.dof_names.index(name)
                for name in self._motion_planner.dof_names
            ]]

        target_pose_from_base = observation.get("target_pose_from_base", None)
        if target_pose_from_base is None:
            target_pose = observation.get("target_pose", None)
            if target_pose is None:
                # TODO msg
                raise ValueError("TODO")
            target_pose_from_base = (
                self._articulation.links[self._motion_planner.base_link].pose.inv()
                * target_pose
            )
            
        return self._motion_planner.compute_action({
            "dof_positions": dof_positions,
            "target_pose": target_pose_from_base,
        })



In [368]:
planner = ArticulationPlanner(
    panda,
    base_link="base_link",
    end_link="ee_link",
)

In [398]:
planner.compute_action({
    "target_pose": Pose(p=[.5, .5, .5])
})

Batch mode enable graph is only supported with num_graph_seeds==1


{'dof_names': ['panda_joint1',
  'panda_joint2',
  'panda_joint3',
  'panda_joint4',
  'panda_joint5',
  'panda_joint6',
  'panda_joint7'],
 'dof_positions': tensor([[[ 2.4087e-01,  1.3185e+00,  9.4309e-01, -9.5012e-01,  2.1686e+00,
            1.2601e+00, -2.2215e-01],
          [ 2.4087e-01,  1.3185e+00,  9.4309e-01, -9.5012e-01,  2.1686e+00,
            1.2601e+00, -2.2215e-01],
          [ 2.3186e-01,  1.3157e+00,  9.4436e-01, -9.5913e-01,  2.1688e+00,
            1.2577e+00, -2.2096e-01],
          [ 2.1324e-01,  1.3099e+00,  9.4695e-01, -9.7778e-01,  2.1690e+00,
            1.2528e+00, -2.1847e-01],
          [ 1.8594e-01,  1.3011e+00,  9.5075e-01, -1.0055e+00,  2.1691e+00,
            1.2455e+00, -2.1475e-01],
          [ 1.5083e-01,  1.2894e+00,  9.5560e-01, -1.0413e+00,  2.1689e+00,
            1.2356e+00, -2.0985e-01],
          [ 1.0865e-01,  1.2749e+00,  9.6133e-01, -1.0843e+00,  2.1684e+00,
            1.2234e+00, -2.0385e-01],
          [ 6.0419e-02,  1.2578e+00,  9.6781e

In [391]:
from robotodo.engines.isaac.articulation import ArticulationDriver


class _TODOArticulationDriver(ArticulationDriver):

    # TODO
    async def execute_action(
        self, 
        action: ArticulationAction,
        # TODO
        error_tolerance_position: float = 1e-1,
        error_tolerance_velocity: float = 1e-1,
    ):
        dof_indices = numpy.s_[:]

        dof_names = action.get("dof_names", None)
        if dof_names is not None:
            # TODO support native .index(<batch_dof_names>)
            dof_indices = [
                self._articulation.dof_names.index(dof_name)
                for dof_name in dof_names
            ]

        dof_positions = action.get("dof_positions", None)
        dof_velocities = action.get("dof_velocities", None)

        if dof_positions is None or dof_velocities is None:
            # TODO
            raise NotImplementedError

        n_p, n_timesteps_p, n_dofs_p = numpy.shape(dof_positions)
        n_v, n_timesteps_v, n_dofs_v = numpy.shape(dof_velocities)

        # TODO
        assert n_p == n_v
        assert n_timesteps_p == n_timesteps_v
        assert n_dofs_p == n_dofs_v

        n_timesteps = n_timesteps_p

        for timestep in range(n_timesteps):
            dof_target_positions = self.dof_target_positions
            # TODO torch 
            dof_target_positions[..., dof_indices] = dof_positions[:, timestep, :].to(dof_target_positions.device)
            self.dof_target_positions = dof_target_positions

            dof_target_velocities = self.dof_target_velocities
            # TODO torch 
            dof_target_velocities[..., dof_indices] = dof_velocities[:, timestep, :].to(dof_target_positions.device)
            self.dof_target_velocities = dof_target_velocities

            # TODO use dt for timeout
            async for _ in self._articulation._scene.on_step:
                position_err = (
                    self._articulation.dof_positions[..., dof_indices] 
                        - self.dof_target_positions[..., dof_indices]
                )
                velocity_err = (
                    self._articulation.dof_velocities[..., dof_indices] 
                        - self.dof_target_velocities[..., dof_indices]
                )

                # TODO
                # print(position_err, velocity_err)

                if all([
                    numpy.allclose(position_err, 0., atol=error_tolerance_position),
                    numpy.allclose(velocity_err, 0., atol=error_tolerance_velocity),
                ]):
                    break



In [392]:
todo = _TODOArticulationDriver(panda)
planner = ArticulationPlanner(
    panda,
    base_link="base_link",
    end_link="ee_link",
)

In [ ]:
# TODO test

action = planner.compute_action({
    "target_pose": Pose(p=[.6, .5, .5])
})

await todo.execute_action(action, error_tolerance_velocity=1)

Batch mode enable graph is only supported with num_graph_seeds==1


In [406]:
panda.links["ee_link"].pose

Pose(p=array([[0.60082304, 0.49896824, 0.49740431]]), q=array([[-2.98483158e-04,  3.01783230e-03, -9.51581039e-04,
         9.99994949e-01]]))

In [402]:
scene.viewer.draw_pose(panda.links["base_link"].pose)


In [403]:
panda.links["base_link"].pose

Pose(p=array([[0.        , 0.46173787, 0.        ]]), q=array([[0., 0., 0., 1.]]))

In [405]:
scene.viewer.draw_pose(Pose(p=[.6, .5, .5]))


In [404]:
scene.viewer.draw_pose(panda.links["base_link"].pose.inv() * Pose(p=[.6, .5, .5]))


In [375]:
panda.driver.dof_target_positions.device

device(type='cpu')

In [373]:
action

{'dof_names': ['panda_joint1',
  'panda_joint2',
  'panda_joint3',
  'panda_joint4',
  'panda_joint5',
  'panda_joint6',
  'panda_joint7'],
 'dof_positions': tensor([[[-1.4790e-04,  2.3591e-04, -6.4824e-05, -6.9801e-02, -1.4219e-04,
           -6.2174e-04,  1.1061e-05],
          [-1.4790e-04,  2.3591e-04, -6.4824e-05, -6.9801e-02, -1.4219e-04,
           -6.2174e-04,  1.1061e-05],
          [ 1.2463e-04,  7.3332e-03,  6.2655e-03, -7.9934e-02,  1.4120e-02,
            8.8178e-03, -3.0314e-03],
          [ 6.8921e-04,  2.2034e-02,  1.9378e-02, -1.0006e-01,  4.3371e-02,
            2.7217e-02, -9.3334e-03],
          [ 1.5278e-03,  4.3871e-02,  3.8855e-02, -1.2888e-01,  8.6337e-02,
            5.3040e-02, -1.8694e-02],
          [ 2.6181e-03,  7.2263e-02,  6.4177e-02, -1.6526e-01,  1.4151e-01,
            8.5000e-02, -3.0864e-02],
          [ 3.9372e-03,  1.0662e-01,  9.4816e-02, -2.0830e-01,  2.0737e-01,
            1.2214e-01, -4.5588e-02],
          [ 5.4623e-03,  1.4633e-01,  1.3024e

In [348]:
_ = """

panda.links

planner = panda.planner(
    base_link="base_link",
    end_link="end_link",
)
planner.compute_action()

"""

In [349]:
# panda.planner.base_link
# panda.planner.compute_action()

In [350]:
planner = MotionPlanner(panda, base_link="base_link", end_link="ee_link")

In [354]:
planner.observation_spec.random()

{'dof_positions': array([[0.22829346, 0.43306016, 0.67703972, 0.84478154, 0.63088692,
         0.01073239, 0.22823491]])}

In [351]:
planner.joints

['panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7']

In [346]:
planner.dof_names

['panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7']

In [ ]:
from curobo.types.robot import RobotConfig as CuroboRobotConfig
from curobo.types.math import Pose as CuroboPose
from curobo.types.robot import JointState as CuroboJointState
from curobo.cuda_robot_model.cuda_robot_generator import (
    CudaRobotGenerator,
    CudaRobotGeneratorConfig as CuroboCudaRobotGeneratorConfig,
)
from curobo.cuda_robot_model.cuda_robot_model import (
    CudaRobotModelConfig as CuroboCudaRobotModelConfig,
)
from curobo.wrap.reacher.motion_gen import (
    MotionGen as CuroboMotionGen,
    MotionGenConfig as CuroboMotionGenConfig,
)



# TODO https://github.com/NVlabs/curobo/blob/ebb71702f3f70e767f40fd8e050674af0288abe8/src/curobo/cuda_robot_model/cuda_robot_generator.py#L319
curobo_motiongen = CuroboMotionGen(
    CuroboMotionGenConfig.load_from_robot_config(
        robot_cfg=CuroboRobotConfig(
            kinematics=CuroboCudaRobotModelConfig.from_config(
                CuroboCudaRobotGeneratorConfig(
                    # TODO infer from dof_names
                    base_link=panda.links["panda_link0"].path[0],
                    ee_link=panda.links["panda_hand"].path[0],
                    custom_kinematics_parser=_CuroboKinematicsParser(
                        list(panda.joints.values())
                    ),
                )
            ),
        ),
    ),
)


# TODO
# curobo_motiongen_basic = CuroboMotionGen(
#     CuroboMotionGenConfig.load_from_robot_config(
#         robot_cfg=CuroboRobotConfig.from_basic(
#             urdf_path='/home/sysadmin/lab/robotodo/__todo__/todo-misc/todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf',
#             base_link="panda_link0",
#             ee_link="panda_link7",
#         ),
#         # world_model=WorldConfig(
#         #     # TODO
#         #     # mesh=curobo_meshes,
#         # ),
#         # interpolation_dt=1 / 250,
#         num_trajopt_seeds=1,
#         num_graph_seeds=1,
#     )
# )




from robotodo.algos.motion_planning.reach import ReachPlanner

planner = ReachPlanner(curobo_motiongen)
# planner = ReachPlanner(curobo_motiongen_basic)
# planner.observation_spec



Batch mode enable graph is only supported with num_graph_seeds==1


In [334]:
curobo_motiongen.joint_names

['/_01/joints/panda_joint1',
 '/_01/joints/panda_joint2',
 '/_01/joints/panda_joint3',
 '/_01/joints/panda_joint4',
 '/_01/joints/panda_joint5',
 '/_01/joints/panda_joint6',
 '/_01/joints/panda_joint7']

In [ ]:
# panda.planner.link_names
# panda.planner.joint_names

In [312]:
curobo_motiongen.joint_names

['/_01/joints/panda_joint1',
 '/_01/joints/panda_joint2',
 '/_01/joints/panda_joint3',
 '/_01/joints/panda_joint4',
 '/_01/joints/panda_joint5',
 '/_01/joints/panda_joint6',
 '/_01/joints/panda_joint7']

In [143]:
dof_positions = panda.dof_positions[..., [
    panda.dof_names.index(name)
    for name in (
        'panda_joint1',
        'panda_joint2',
        'panda_joint3',
        'panda_joint4',
        'panda_joint5',
        'panda_joint6',
        'panda_joint7',
    )
]]


# TODO
target_pose = panda.links["panda_link0"].pose.inv() * Pose([.5, .5, .5]) # panda.links["panda_link7"].pose

action, _ = planner.compute_action({
    "dof_positions": dof_positions,
    "target_pose": target_pose,
}, include_raw_result=True)

Batch mode enable graph is only supported with num_graph_seeds==1


In [ ]:
# TODO !!! .iter instead
for pos, vel in zip(action["dof_positions"][0], action["dof_velocities"][0]):
    dof_target_pos = panda.driver.dof_target_positions
    dof_target_pos[..., :-2] = pos
    panda.driver.dof_target_positions = dof_target_pos

    dof_target_vel = panda.driver.dof_target_velocities
    dof_target_vel[..., :-2] = vel
    panda.driver.dof_target_velocities = dof_target_vel    

    async for _ in scene.on_step:
        pos_err = panda.dof_positions - panda.driver.dof_target_positions
        vel_err = panda.dof_velocities - panda.driver.dof_target_velocities
        
        if all([
            numpy.allclose(pos_err[..., :-2], 0, atol=1e-1),
            numpy.allclose(vel_err[..., :-2], 0, atol=1),
        ]):
            break

        print("...", pos_err, vel_err)

In [122]:
pos_err

tensor([[-3.1817e-04,  3.1145e-03, -3.0245e-03,  1.7643e-05,  5.2714e-04,
          2.9298e-03,  3.2622e-04,  2.1833e-03,  1.5810e-07]])

In [119]:
panda.driver.dof_target_velocities

tensor([[-3.7785e-02,  1.6510e-02,  5.0692e-04,  8.5455e-04,  2.7623e-05,
         -3.3425e-04,  3.1514e-02,  0.0000e+00,  0.0000e+00]])

In [73]:
panda.dof_positions

tensor([[ 5.8024e-01, -4.6349e-01, -5.6984e-01, -2.5976e+00,  2.8446e+00,
          9.6707e-01,  5.5106e-01,  6.8863e-05,  1.1052e-06]])

In [64]:
panda.dof_names

['panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7',
 'panda_finger_joint1',
 'panda_finger_joint2']

In [63]:
target_pose

Pose(p=array([[0.5, 0.5, 0.5]]), q=array([[0., 0., 0., 1.]]))

In [45]:
curobo_motiongen.joint_names

['/_01/joints/panda_joint1',
 '/_01/joints/panda_joint2',
 '/_01/joints/panda_joint3',
 '/_01/joints/panda_joint4',
 '/_01/joints/panda_joint5',
 '/_01/joints/panda_joint6',
 '/_01/joints/panda_joint7']

In [44]:
panda.links["base_link"].pose.inv() * Pose([.5, .5, .5])

Pose(p=array([[0.5, 0.5, 0.5]]), q=array([[0., 0., 0., 1.]]))

In [269]:
curobo_motiongen.joint_names

['/Ant/joints/right_back_leg', '/Ant/joints/right_back_foot']

In [222]:
panda.links["base_link"].path[0]

'/_01/base_link'

['panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7',
 'panda_finger_joint1',
 'panda_finger_joint2']

In [218]:
panda.dof_positions


tensor([[-1.5290e-04,  2.3602e-04, -7.0062e-05, -6.9801e-02, -1.4394e-04,
         -6.2159e-04,  1.1547e-05,  6.1375e-04,  0.0000e+00]])

In [214]:
curobo_motiongen.joint_names


['/Ant/joints/right_back_leg', '/Ant/joints/right_back_foot']

In [ ]:
from curobo.types.robot import RobotConfig as CuroboRobotConfig
from curobo.types.math import Pose as CuroboPose
from curobo.types.robot import JointState as CuroboJointState
from curobo.cuda_robot_model.cuda_robot_generator import (
    CudaRobotGenerator,
    CudaRobotGeneratorConfig as CuroboCudaRobotGeneratorConfig,
)
from curobo.cuda_robot_model.cuda_robot_model import (
    CudaRobotModelConfig as CuroboCudaRobotModelConfig,
)
from curobo.wrap.reacher.motion_gen import (
    MotionGen as CuroboMotionGen,
    MotionGenConfig as CuroboMotionGenConfig,
)


# TODO https://github.com/NVlabs/curobo/blob/ebb71702f3f70e767f40fd8e050674af0288abe8/src/curobo/cuda_robot_model/cuda_robot_generator.py#L319
curobo_motiongen = CuroboMotionGen(
    CuroboMotionGenConfig.load_from_robot_config(
        robot_cfg=CuroboRobotConfig(
            kinematics=CuroboCudaRobotModelConfig.from_config(
                CuroboCudaRobotGeneratorConfig(
                    # TODO
                    base_link=ant.links["torso"].path[0],
                    ee_link=ant.links["right_back_foot"].path[0],
                    custom_kinematics_parser=_CuroboKinematicsParser(
                        list(ant.joints.values())
                    ),
                )
            ),
        ),
    ),
)
curobo_motiongen.joint_names
from robotodo.algos.motion_planning.reach import ReachPlanner

planner = ReachPlanner(curobo_motiongen)
planner.observation_spec
dof_positions = ant.dof_positions[..., [
    ant.dof_names.index("right_back_leg"), 
    ant.dof_names.index("right_back_foot"),
]]

# TODO
target_pose = ant.links["torso"].pose.inv() * ant.links["right_back_foot"].pose
target_pose *= Pose(q=[0, 0.05, 0, 1])

planner.compute_action({
    "dof_positions": dof_positions,
    "target_pose": target_pose,
}, include_raw_result=True)

In [212]:
scene.viewer.clear_drawings()
# scene.viewer.draw_pose(ant.links["torso"].pose.inv() * ant.links["right_back_foot"].pose)
# scene.viewer.draw_pose(ant.links["right_back_foot"].pose * ant.links["torso"].pose.inv())

In [181]:
ant.links["torso"].pose.inv() * ant.links["right_back_foot"].pose

Pose(p=array([[ 4.81763889e-01, -2.24680018e-01, -1.07731823e-07]]), q=array([[0.10938928, 0.2345545 , 0.33032086, 0.90769285]]))

In [182]:
ant.links["right_back_foot"].pose * ant.links["torso"].pose.inv()

Pose(p=array([[-0.43273398, -0.56065113,  0.25851738]]), q=array([[0.07424629, 0.1243053 , 0.39386467, 0.90769285]]))

In [176]:
target_pose

Pose(p=array([[ 0.4548649 , -0.28921747,  0.03195063]]), q=array([[0.31360319, 0.43309521, 0.13352942, 0.83441685]]))

In [179]:
scene.viewer.draw_pose(ant.links["right_back_foot"].pose)

Pose(p=array([[ 3.99999946e-01, -3.99999976e-01,  4.47034836e-08]]), q=array([[0., 0., 0., 1.]]))

Pose(p=array([[ 3.99999946e-01, -3.99999976e-01,  4.47034836e-08]]), q=array([[0., 0., 0., 1.]]))

In [152]:
ant.dof_positions

tensor([[0., 0., 0., 0., 0., 0., 0., 0.]])

In [154]:
ant.dof_positions[..., [
    ant.dof_names.index("right_back_leg"), 
    ant.dof_names.index("right_back_foot"),
]]

tensor([[0.6981, 0.5236]])

In [156]:
ant.links

{'torso': <robotodo.engines.isaac.entity.Entity at 0x7f8470279d90>,
 'front_left_leg': <robotodo.engines.isaac.entity.Entity at 0x7f84786b7e10>,
 'front_right_leg': <robotodo.engines.isaac.entity.Entity at 0x7f8470158cd0>,
 'left_back_leg': <robotodo.engines.isaac.entity.Entity at 0x7f847015a290>,
 'right_back_leg': <robotodo.engines.isaac.entity.Entity at 0x7f8470158510>,
 'front_left_foot': <robotodo.engines.isaac.entity.Entity at 0x7f8470158bd0>,
 'front_right_foot': <robotodo.engines.isaac.entity.Entity at 0x7f847015a2d0>,
 'left_back_foot': <robotodo.engines.isaac.entity.Entity at 0x7f84701593d0>,
 'right_back_foot': <robotodo.engines.isaac.entity.Entity at 0x7f8470158b50>}

In [146]:
ant.joints

{'front_left_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84705de0d0>,
 'front_right_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f847822fc50>,
 'left_back_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84780a7e80>,
 'right_back_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84705dd1d0>,
 'front_left_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84705dd450>,
 'front_right_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84705dd680>,
 'left_back_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84705dd3b0>,
 'right_back_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84705dd2c0>}

In [ ]:
curobo_motiongen.plan_batch()

MotionGen(ik_seeds=32, graph_seeds=4, trajopt_seeds=4, noisy_trajopt_seeds=1, batch_ik_seeds=32, batch_trajopt_seeds=1, robot_cfg=RobotConfig(kinematics=CudaRobotModelConfig(tensor_args=TensorDeviceType(device=device(type='cuda', index=0), dtype=torch.float32, collision_geometry_dtype=torch.float32, collision_gradient_dtype=torch.float32, collision_distance_dtype=torch.float32), link_names=['/Ant/right_back_foot'], kinematics_config=KinematicsTensorConfig(fixed_transforms=tensor([[[ 1.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 1.0000,  0.0000,  0.0000,  0.2000],
         [ 0.0000,  1.0000,  0.0000, -0.2000],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 1.0000,  0.0000,  0.0000,  0.2000],
         [ 0.0000,  1.0000,  0.0000, -0.2000],
         [ 0.0000,  0.0000,  1.0000,  0.0000],
  

In [127]:
ant.joints

{'front_left_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84781bb070>,
 'front_right_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84781ba1c0>,
 'left_back_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84781bb250>,
 'right_back_leg': <robotodo.engines.isaac.articulation.Joint at 0x7f84781ba940>,
 'front_left_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84781bb390>,
 'front_right_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84781baa30>,
 'left_back_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84781ba8a0>,
 'right_back_foot': <robotodo.engines.isaac.articulation.Joint at 0x7f84781ba7b0>}

In [ ]:
todo = _CuroboKinematicsParser(
                        list(ant.joints.values())
                    )
todo._build_link_parameters()

{'/Ant/front_left_leg': LinkParams(link_name='/Ant/front_left_leg', joint_name='/Ant/joints/front_left_leg', joint_type=<JointType.X_ROT: 3>, fixed_transform=array([[1. , 0. , 0. , 0.2],
        [0. , 1. , 0. , 0.2],
        [0. , 0. , 1. , 0. ],
        [0. , 0. , 0. , 1. ]]), parent_link_name='/Ant/torso', joint_limits=array([-40.,  40.]), joint_axis=None, joint_id=None, joint_velocity_limits=[-2.0, 2.0], joint_offset=[1.0, 0.0], mimic_joint_name=None),
 '/Ant/front_right_leg': LinkParams(link_name='/Ant/front_right_leg', joint_name='/Ant/joints/front_right_leg', joint_type=<JointType.X_ROT: 3>, fixed_transform=array([[ 1. ,  0. ,  0. , -0.2],
        [ 0. ,  1. ,  0. ,  0.2],
        [ 0. ,  0. ,  1. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ]]), parent_link_name='/Ant/torso', joint_limits=array([-40.,  40.]), joint_axis=None, joint_id=None, joint_velocity_limits=[-2.0, 2.0], joint_offset=[1.0, 0.0], mimic_joint_name=None),
 '/Ant/left_back_leg': LinkParams(link_name='/Ant/left_back_l

In [129]:
todo._parent_map

{'/Ant/front_left_leg': {'parent': '/Ant/torso'},
 '/Ant/front_right_leg': {'parent': '/Ant/torso'},
 '/Ant/left_back_leg': {'parent': '/Ant/torso'},
 '/Ant/right_back_leg': {'parent': '/Ant/torso'},
 '/Ant/front_left_foot': {'parent': '/Ant/front_left_leg'},
 '/Ant/front_right_foot': {'parent': '/Ant/front_right_leg'},
 '/Ant/left_back_foot': {'parent': '/Ant/left_back_leg'},
 '/Ant/right_back_foot': {'parent': '/Ant/right_back_leg'}}

In [79]:
scene.viewer.selected_entity

In [95]:
ant.joints["front_left_leg"].kind

array([1])

In [79]:
ant._isaac_physics_articulation_view.shared_metatype.dof_names

['front_left_leg:0',
 'front_left_leg:1',
 'front_left_leg:2',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg',
 'front_left_foot',
 'front_right_foot',
 'left_back_foot',
 'right_back_foot']

In [44]:
ant._isaac_physics_articulation_view.shared_metatype.dof_count

10

In [45]:
ant._scene._isaac_physx.reset_simulation()

In [35]:
import omni

omni.physics.tensors.reset()


In [74]:
(ant._isaac_physics_articulation_view.shared_metatype.dof_types)

[<DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>,
 <DofType.Rotation: 0>]

In [87]:
from omni.physics.tensors import JointType

JointType?

Docstring:     
Joint type.

Members:

  Invalid

  Fixed

  Revolute

  Prismatic

  Spherical
Init docstring: __init__(self: omni.physics.tensors.bindings._physicsTensors.JointType, value: int) -> None
File:           ~/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/exts/3/omni.physics.tensors-107.3.18+107.3.1.lx64.r.cp311.u353/omni/physics/tensors/bindings/_physicsTensors.cpython-311-x86_64-linux-gnu.so
Type:           pybind11_type
Subclasses:     

In [89]:
JointType.Fixed

<JointType.Fixed: 0>

2025-11-11T01:35:33Z [68,109,270ms] [Error] [omni.kit.manipulator.prim.core.model] Traceback (most recent call last):
  File "/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/exts/3/omni.kit.manipulator.prim.core-107.0.8+8131b85d/omni/kit/manipulator/prim/core/model.py", line 1275, in _update_transform_from_prims_async
    pivot_prim_path = self._pivot_prim.GetPath()
                      ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'GetPath'



In [15]:
from omni.physics.tensors import DofType

DofType?

Docstring:     
DOF type.

Members:

  Invalid

  Rotation

  Translation
Init docstring: __init__(self: omni.physics.tensors.bindings._physicsTensors.DofType, value: int) -> None
File:           ~/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/exts/3/omni.physics.tensors-107.3.18+107.3.1.lx64.r.cp311.u353/omni/physics/tensors/bindings/_physicsTensors.cpython-311-x86_64-linux-gnu.so
Type:           pybind11_type
Subclasses:     

In [69]:
scene._isaac_physx.reset_simulation()

In [78]:
ant.dof_names

['front_left_leg',
 'front_right_leg',
 'left_back_leg',
 'right_back_leg',
 'front_left_foot',
 'front_right_foot',
 'left_back_foot',
 'right_back_foot']

In [77]:
ant.dof_position_limits

tensor([[[-0.6981,  0.6981],
         [-0.6981,  0.6981],
         [-0.6981,  0.6981],
         [-0.6981,  0.6981],
         [ 0.5236,  1.7453],
         [-1.7453, -0.5236],
         [-1.7453, -0.5236],
         [ 0.5236,  1.7453]]])

In [27]:
from robotodo.engines.isaac.articulation import Joint

In [31]:
Joint("/Ant/joints/*", scene=scene).body0.path


['/Ant/torso',
 '/Ant/front_left_leg',
 '/Ant/torso',
 '/Ant/front_right_leg',
 '/Ant/torso',
 '/Ant/left_back_leg',
 '/Ant/torso',
 '/Ant/right_back_leg']

In [51]:
Joint("/Ant/joints/*", scene=scene).body1.path


['/Ant/front_left_leg',
 '/Ant/front_left_foot',
 '/Ant/front_right_leg',
 '/Ant/front_right_foot',
 '/Ant/left_back_leg',
 '/Ant/left_back_foot',
 '/Ant/right_back_leg',
 '/Ant/right_back_foot']

In [ ]:
Joint("/Ant/joints/*", scene=scene)._usd_prims

[Usd.Prim(</Ant/joints/front_left_leg>),
 Usd.Prim(</Ant/joints/front_left_foot>),
 Usd.Prim(</Ant/joints/front_right_leg>),
 Usd.Prim(</Ant/joints/front_right_foot>),
 Usd.Prim(</Ant/joints/left_back_leg>),
 Usd.Prim(</Ant/joints/left_back_foot>),
 Usd.Prim(</Ant/joints/right_back_leg>),
 Usd.Prim(</Ant/joints/right_back_foot>)]

In [40]:
Joint("/Ant/joints/*", scene=scene).pose_in_body0

Pose(p=array([[ 0.2,  0.2,  0. ],
       [ 0.2,  0.2,  0. ],
       [-0.2,  0.2,  0. ],
       [-0.2,  0.2,  0. ],
       [-0.2, -0.2,  0. ],
       [-0.2, -0.2,  0. ],
       [ 0.2, -0.2,  0. ],
       [ 0.2, -0.2,  0. ]]), q=array([[ 0.        , -0.70710683,  0.        ,  0.70710683],
       [ 0.        ,  0.        ,  0.92387962,  0.38268334],
       [ 0.        , -0.70710683,  0.        ,  0.70710683],
       [ 0.        ,  0.        ,  0.38268346,  0.92387956],
       [ 0.        , -0.70710683,  0.        ,  0.70710683],
       [ 0.        ,  0.        ,  0.92387962,  0.38268334],
       [ 0.        , -0.70710683,  0.        ,  0.70710683],
       [ 0.        ,  0.        ,  0.38268346,  0.92387956]]))

In [ ]:
# ant.joints[]

In [53]:
import numpy

numpy.asarray(ant._isaac_physics_articulation_view.dof_paths)[..., 4]

array(['/Ant/joints/left_back_leg'], dtype='<U28')

In [48]:
ant._isaac_physics_articulation_view.shared_metatype.joint_indices

{'front_left_foot': 4,
 'front_left_leg': 0,
 'front_right_foot': 5,
 'front_right_leg': 1,
 'left_back_foot': 6,
 'left_back_leg': 2,
 'right_back_foot': 7,
 'right_back_leg': 3}

In [56]:
ant.dof_position_limits.shape

torch.Size([1, 10, 2])

In [57]:
ant._isaac_physics_articulation_view.shared_metatype.dof_indices

{'front_left_foot': 6,
 'front_left_leg:0': 0,
 'front_left_leg:1': 1,
 'front_left_leg:2': 2,
 'front_right_foot': 7,
 'front_right_leg': 3,
 'left_back_foot': 8,
 'left_back_leg': 4,
 'right_back_foot': 9,
 'right_back_leg': 5}

[['/Ant/joints/front_left_leg',
  '/Ant/joints/front_right_leg',
  '/Ant/joints/left_back_leg',
  '/Ant/joints/right_back_leg',
  '/Ant/joints/front_left_foot',
  '/Ant/joints/front_right_foot',
  '/Ant/joints/left_back_foot',
  '/Ant/joints/right_back_foot']]

In [29]:
todo_joint_paths = set(ant._isaac_physics_articulation_view.dof_paths[0])
todo_joint_paths

{'/Ant/joints/front_left_foot',
 '/Ant/joints/front_left_leg',
 '/Ant/joints/front_right_foot',
 '/Ant/joints/front_right_leg',
 '/Ant/joints/left_back_foot',
 '/Ant/joints/left_back_leg',
 '/Ant/joints/right_back_foot',
 '/Ant/joints/right_back_leg'}

In [ ]:


todo_joint_paths = set(ant._isaac_physics_articulation_view.dof_paths[0])

for p in todo_joint_paths:
    api = pxr.UsdPhysics.Joint.Get(scene._usd_stage, p)
    print(api, (api.GetBody0Rel().GetTargets(), api.GetBody1Rel().GetTargets()), api.GetLocalPos0Attr().Get(), api.GetLocalRot0Attr().Get())

UsdPhysics.Joint(Usd.Prim(</Ant/joints/front_right_leg>)) ([Sdf.Path('/Ant/torso')], [Sdf.Path('/Ant/front_right_leg')]) (-0.2, 0.2, 0) (0.7071068, 0, -0.7071068, 0)
UsdPhysics.Joint(Usd.Prim(</Ant/joints/right_back_leg>)) ([Sdf.Path('/Ant/torso')], [Sdf.Path('/Ant/right_back_leg')]) (0.2, -0.2, 0) (0.7071068, 0, -0.7071068, 0)
UsdPhysics.Joint(Usd.Prim(</Ant/joints/right_back_foot>)) ([Sdf.Path('/Ant/right_back_leg')], [Sdf.Path('/Ant/right_back_foot')]) (0.2, -0.2, 0) (0.92387956, 0, 0, 0.38268346)
UsdPhysics.Joint(Usd.Prim(</Ant/joints/left_back_leg>)) ([Sdf.Path('/Ant/torso')], [Sdf.Path('/Ant/left_back_leg')]) (-0.2, -0.2, 0) (0.7071068, 0, -0.7071068, 0)
UsdPhysics.Joint(Usd.Prim(</Ant/joints/front_right_foot>)) ([Sdf.Path('/Ant/front_right_leg')], [Sdf.Path('/Ant/front_right_foot')]) (-0.2, 0.2, 0) (0.92387956, 0, 0, 0.38268346)
UsdPhysics.Joint(Usd.Prim(</Ant/joints/left_back_foot>)) ([Sdf.Path('/Ant/left_back_leg')], [Sdf.Path('/Ant/left_back_foot')]) (-0.2, -0.2, 0) (0.382683

In [52]:
api.GetLocalPos0Attr().Get()

Gf.Vec3f(0.20000000298023224, 0.20000000298023224, 0.0)

In [32]:
rel = api.GetBody0Rel()
rel.GetTargets()

[Sdf.Path('/Ant/torso')]

In [ ]:

# TODO USDJointHelper 


In [ ]:
panda._isaac_physics_articulation_view.shared_metatype.joint_names


['panda_fixed',
 'panda_joint1',
 'panda_joint2',
 'panda_joint3',
 'panda_joint4',
 'panda_joint5',
 'panda_joint6',
 'panda_joint7',
 'panda_joint8',
 'right_gripper',
 'panda_hand_joint',
 'ee_fixed_joint',
 'panda_finger_joint1',
 'panda_finger_joint2']

In [30]:
panda._isaac_physics_articulation_view.shared_metatype.joint_types

[<JointType.Fixed: 0>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Revolute: 1>,
 <JointType.Fixed: 0>,
 <JointType.Fixed: 0>,
 <JointType.Fixed: 0>,
 <JointType.Fixed: 0>,
 <JointType.Prismatic: 2>,
 <JointType.Prismatic: 2>]

In [66]:

import os
import yaml

prefix = "todo-curobo-sample-configs/franka"
with open("todo-curobo-sample-configs/franka/config.yml") as f:
    config_file_content = yaml.load(f, yaml.FullLoader)


for key in [
    "urdf_path",
    "asset_root_path",
    "collision_spheres",
]:
    config_file_content["robot_cfg"]["kinematics"][key] = os.path.join(
        os.path.abspath(prefix),
        config_file_content["robot_cfg"]["kinematics"][key],
    )

config_file_content

{'robot_cfg': {'kinematics': {'urdf_path': '/home/sysadmin/lab/robotodo/__todo__/todo-misc/todo-curobo-sample-configs/franka/franka_description/franka_panda.urdf',
   'asset_root_path': '/home/sysadmin/lab/robotodo/__todo__/todo-misc/todo-curobo-sample-configs/franka/franka_description',
   'base_link': 'panda_link0',
   'ee_link': 'panda_hand',
   'collision_link_names': ['panda_link0',
    'panda_link1',
    'panda_link2',
    'panda_link3',
    'panda_link4',
    'panda_link5',
    'panda_link6',
    'panda_link7',
    'panda_hand',
    'panda_leftfinger',
    'panda_rightfinger',
    'attached_object'],
   'collision_spheres': '/home/sysadmin/lab/robotodo/__todo__/todo-misc/todo-curobo-sample-configs/franka/franka_mesh.yml',
   'collision_sphere_buffer': 0.004,
   'extra_collision_spheres': {'attached_object': 4},
   'use_global_cumul': True,
   'self_collision_ignore': {'panda_link0': ['panda_link1', 'panda_link2'],
    'panda_link1': ['panda_link2', 'panda_link3', 'panda_link4'],

In [67]:
from robotodo.algos.motion_planning.reach import ReachPlanner

planner = ReachPlanner(config_file_content)

In [68]:
planner.compute_action

<bound method ReachPlanner.compute_action of <robotodo.algos.motion_planning.reach.ReachPlanner object at 0x7fae38254990>>

In [69]:
_ = """


panda.planner.link_names
action = panda.planner.compute_action(
    link_name=..., 
    target_pose=...,
    num_timesteps=...,
)

await panda.driver.execute_action(
    action,
    # TODO options
)

"""

In [70]:
# panda.planner.compute_action(link=..., target_pose=...)
# panda.driver.execute_action()

In [ ]:
from tensorspecs import TensorTableSpec, TensorSpec, TensorTableLike


# TODO batch
action_spec = TensorTableSpec({
    "dof_positions": TensorSpec("time n? dof"),
    "dof_velocities": TensorSpec("time n? dof"),
})


class ArticulationDriver:

    async def execute_action(self, action: TensorTableLike[action_spec]):
        pass

In [71]:
import numpy
from robotodo.utils.pose import Pose

action = planner.compute_action({
    "dof_positions": panda.dof_positions[..., :-2],
    "target_pose": panda.root_pose.inv() * Pose(p=[.4, -0, .6], q=[0, 0, 0, 1]),
    # "target_pose": panda.root_pose.inv() * Pose(p=[.5, .5, .5], q=[0, 0, 0, 1]),
})

# TODO !!! .iter instead
for pos, vel in zip(action["dof_positions"][0], action["dof_velocities"][0]):
    dof_target_pos = panda.driver.dof_target_positions
    dof_target_pos[..., :-2] = pos
    panda.driver.dof_target_positions = dof_target_pos

    dof_target_vel = panda.driver.dof_target_velocities
    dof_target_vel[..., :-2] = vel
    panda.driver.dof_target_velocities = dof_target_vel    

    async for event in scene.on_step:
        pos_err = panda.dof_positions - panda.driver.dof_target_positions
        vel_err = panda.dof_velocities - panda.driver.dof_target_velocities
        
        if all([
            numpy.allclose(pos_err[..., :-2], 0, atol=1e-1),
            # numpy.allclose(vel_err, 0, atol=1e-1),
        ]):
            break

    # print("...", pos_err, vel_err)

Batch mode enable graph is only supported with num_graph_seeds==1


In [ ]:
panda.link_names

['base_link',
 'panda_link0',
 'panda_link1',
 'panda_link2',
 'panda_link3',
 'panda_link4',
 'panda_link5',
 'panda_link6',
 'panda_link7',
 'panda_link8',
 'right_gripper',
 'panda_hand',
 'ee_link',
 'panda_leftfinger',
 'panda_rightfinger']

In [6]:

# end_link = ...
# fingertip_link = ...

import numpy as np

# TODO
def standoff_from_poses(T_ee, T_tip, approach_dir_g=np.array([0,0,1]), take_abs=True):
    """
    T_ee: (4,4) EE/TCP pose in world
    T_tips: list of (4,4) fingertip poses in world (one or two)
    approach_dir_g: approach axis in EE frame (e.g., [0,0,1])
    """
    R_ee = T_ee[..., :3, :3]
    p_ee = T_ee[..., :3, 3]
    a_w = R_ee @ (approach_dir_g / np.linalg.norm(approach_dir_g))

    p_tip = T_tip[..., :3, 3]
    return (p_tip - p_ee) @ a_w.T
    # return abs(s) if take_abs else s

# TODO names !!!!
gripper_approach_direction = numpy.array([0, 0, 1])
gripper_standoff_fingertips = standoff_from_poses(
    # TODO
    # panda.link_poses[..., panda.link_names.index("panda_link8")].to_matrix(),
    # panda.link_poses[..., panda.link_names.index("panda_leftfinger")].to_matrix(),
    panda.link_poses[..., panda.link_names.index("panda_hand")].to_matrix(),
    panda.link_poses[..., panda.link_names.index("ee_link")].to_matrix(),
    approach_dir_g=gripper_approach_direction,
)
gripper_standoff_fingertips = numpy.squeeze(
    gripper_standoff_fingertips
)
gripper_standoff_fingertips



import numpy as np

def _unit(v): 
    v = np.asarray(v, float)
    return v / (np.linalg.norm(v))

def compute_gripper_aperture(T_tipL, T_tipR, T_ee,
                     approach_g=(0,0,1), align_g=(0,1,0)):
    """
    Compute parallel-jaw aperture from two fingertip transforms.
    
    T_tipL, T_tipR, T_ee: 4x4 world transforms
    approach_g: approach axis in EE frame (default z)
    align_g: alignment axis in EE frame (default y)
    """
    R_ee = T_ee[..., :3,:3]
    pL, pR = T_tipL[..., :3,3], T_tipR[..., :3,3]

    # separation axis = cross(approach, align), expressed in world
    sep_w = R_ee @ _unit(np.cross(approach_g, align_g))

    return ((pR - pL) @ sep_w.T)



gripper_align_axis = numpy.array([0, 1, 0])

# TODO NOTE ensure open gripper
gripper_maximum_aperture = compute_gripper_aperture(
    panda.link_poses[..., panda.link_names.index("panda_leftfinger")].to_matrix(),
    panda.link_poses[..., panda.link_names.index("panda_rightfinger")].to_matrix(),
    T_ee=panda.link_poses[..., panda.link_names.index("panda_link8")].to_matrix(),
    # T_ee=panda.link_poses[..., panda.link_names.index("panda_hand")].to_matrix(),
    approach_g=gripper_approach_direction,
    align_g=gripper_align_axis,
)
gripper_maximum_aperture = numpy.squeeze(gripper_maximum_aperture)

gripper_standoff_fingertips, gripper_maximum_aperture

(array(0.09999994), array(4.68832306e-05))

In [8]:

viewer = scene.viewer
viewer.draw_pose(panda.link_poses[..., panda.link_names.index("panda_leftfinger")])
viewer.draw_pose(panda.link_poses[..., panda.link_names.index("panda_rightfinger")])
viewer.draw_pose(panda.link_poses[..., panda.link_names.index("panda_hand")])


In [ ]:
panda.link_poses[..., panda.link_names.index("panda_leftfinger")].inv() * panda.link_poses[..., panda.link_names.index("panda_rightfinger")]

Pose(p=array([[ 5.69875122e-09, -8.00000704e-02, -6.15182608e-08]]), q=array([[-2.44053228e-08,  2.32026833e-10, -4.13094508e-09,
         1.00000000e+00]]))

In [12]:
import importlib
importlib.invalidate_caches()

In [15]:
import numpy as np
from sklearn.neighbors import BallTree

# 1. Prepare your data
# Create a 2D dataset (e.g., points in a plane)
X = np.random.rand(100, 3) * 100  # 100 points, 2 dimensions, values up to 100

# 2. Construct the Ball Tree
# The metric parameter defines how distances are calculated (e.g., 'euclidean', 'haversine' for spherical data)
tree = BallTree(X, metric='euclidean')

# 3. Perform queries (e.g., nearest neighbors)
# Find the 5 nearest neighbors for a query point
query_point = np.array([[50, 50, 50]])
distances, indices = tree.query(query_point, k=5)

print("Query Point:", query_point)
print("Distances to 5 nearest neighbors:", distances)
print("Indices of 5 nearest neighbors:", indices)

# 4. Range queries (find all points within a certain radius)
radius = 10
points_in_range_indices = tree.query_radius(query_point, r=radius)

print(f"Points within radius {radius}:", points_in_range_indices)

Query Point: [[50 50 50]]
Distances to 5 nearest neighbors: [[12.3338423  16.38440064 18.89640955 20.46728848 21.25841367]]
Indices of 5 nearest neighbors: [[24 77  6 74 99]]
Points within radius 10: [array([], dtype=int64)]


In [21]:
from robotodo.utils.geometry import export_trimesh
from robotodo.engines.isaac.entity import Entity

todo_trimesh = export_trimesh(
    Entity("/_01/panda_link2", scene=scene).geometry[0][0]
)
todo_trimesh.show()

In [23]:
import trimesh

trimesh.nsphere.minimum_nsphere(todo_trimesh)

(TrackedArray([-0.00046949,  0.0284515 ,  0.07537497]), 0.14504040897056641)

In [25]:
import numpy as np
import trimesh
from trimesh.nsphere import minimum_nsphere
import random

class SphereTreeNode:
    """A node in a sphere tree."""
    def __init__(self, center, radius, triangles=None):
        self.center = center
        self.radius = radius
        self.triangles = triangles
        self.left = None
        self.right = None

def build_sphere_tree(mesh, face_indices, leaf_threshold=10):
    """
    Recursively builds a binary sphere tree from a subset of mesh faces.
    """
    current_faces = mesh.faces[face_indices]
    current_vertices = mesh.vertices[current_faces.flatten()]
    center, radius = minimum_nsphere(current_vertices)
    node = SphereTreeNode(center, radius)
    
    if len(face_indices) <= leaf_threshold:
        node.triangles = face_indices
        return node
    
    bounds = mesh.bounds
    extents = bounds[1] - bounds[0] # Use the bounds of the current subset of vertices for better split
    split_axis = np.argmax(extents)
    
    face_centroids = mesh.triangles_center[face_indices]
    sorted_indices = np.argsort(face_centroids[:, split_axis])
    
    mid_point = len(sorted_indices) // 2
    left_indices = face_indices[sorted_indices[:mid_point]]
    right_indices = face_indices[sorted_indices[mid_point:]]
    
    node.left = build_sphere_tree(mesh, left_indices, leaf_threshold)
    node.right = build_sphere_tree(mesh, right_indices, leaf_threshold)
    
    return node

def collect_spheres_recursive(node, sphere_list, depth, max_depth=5):
    """Traverses the tree and collects sphere meshes up to a certain depth."""
    if node is None or depth > max_depth:
        return
    
    # Create a trimesh Sphere primitive for visualization
    # We can use color to denote depth level
    sphere_vis = trimesh.primitives.Sphere(
        center=node.center, 
        radius=node.radius
    )
    
    # Set a semi-transparent color
    # Colors are RGBA (0-255)
    color = [random.randint(50, 200), random.randint(50, 200), random.randint(50, 200), 50] # Semi-transparent random color
    if node.left is None and node.right is None:
         color = [255, 0, 0, 100] # Red for leaf nodes

    sphere_vis.visual.face_colors = color
    sphere_list.append(sphere_vis)
    
    collect_spheres_recursive(node.left, sphere_list, depth + 1, max_depth)
    collect_spheres_recursive(node.right, sphere_list, depth + 1, max_depth)

# --- Usage Example and Visualization ---

# 1. Load or create a mesh
mesh = todo_trimesh
mesh.process() 

# 2. Build the sphere tree
all_face_indices = np.arange(len(mesh.faces))
sphere_tree_root = build_sphere_tree(mesh, all_face_indices, leaf_threshold=20)

# 3. Collect spheres for visualization
visualization_spheres = []
# Adjust max_depth to visualize more levels of the hierarchy
collect_spheres_recursive(sphere_tree_root, visualization_spheres, depth=0, max_depth=3) 

# 4. Create a scene and add the original mesh and the spheres
scene = trimesh.Scene([mesh])

# Set original mesh to be semi-transparent for better viewing of internal spheres
mesh.visual.face_colors = [100, 100, 240, 100] 

for sphere in visualization_spheres:
    scene.add_geometry(sphere)

# 5. Visualize the scene
# This opens a viewer window
scene.show()


QhullError: QH6022 qhull input error: 2'th dimension's new bounds [-0.5, 0.5] too wide for
existing bounds [0.19, 0.19]

While executing:  | qhull i QbB Pp Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 851183456  incidence  QbBound-unit-box 0.5  Pprecision-ignore
  Qtriangulate  _pre-merge  _zero-centrum  _maxoutside  0
